Vou utilizar os arquivos que estão descritos na pasta `limited_supervision` para cada idioma.

Vou avaliar o tamanho dos áudios concatenados (total dá 1 hora de áudio?) e proporção de áudios masculinos e femininos.

In [1]:
import os
import gc
import pandas as pd
import numpy as np
import glob

import librosa
import librosa.display
import IPython.display as ipd

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
LANGUAGE_PATH = "../data/mls/mls_{}"
IDS_PATH = os.path.join(LANGUAGE_PATH, "train/limited_supervision/1hr/{}") # language, folder
AUDIO_PATH = os.path.join(LANGUAGE_PATH, "train/audio/{}/{}/{}.flac") # language, speaker, book, id
LANGUAGES = ["italian", "polish", "portuguese", "spanish"]
FOLDERS = list(range(0, 6))

In [3]:
FOLDERS

[0, 1, 2, 3, 4, 5]

Vou obter todos os IDs que estão nos arquivos `handles.txt`

In [4]:
def get_content_from_file(filepath):
    with open(filepath) as f:
        lines = f.readlines()
    lines = [line[:-1] for line in lines]
    return lines

In [5]:
#get_content_from_file("../data/mls/mls_italian/train/limited_supervision/1hr/0/handles.txt")

In [6]:
ids_map = pd.DataFrame(columns=["language", "ids"])
for language in LANGUAGES:
    print("Idioma: {}".format(language))
    ids = []
    for folder in FOLDERS:
        print("Folder {}".format(folder))
        data_path = IDS_PATH.format(language, folder)
        content = get_content_from_file(os.path.join(data_path, "handles.txt"))
        ids = ids + content
    ids_map.loc[len(ids_map), :] = [language, ids]
    print()

Idioma: italian
Folder 0
Folder 1
Folder 2
Folder 3
Folder 4
Folder 5

Idioma: polish
Folder 0
Folder 1
Folder 2
Folder 3
Folder 4
Folder 5

Idioma: portuguese
Folder 0
Folder 1
Folder 2
Folder 3
Folder 4
Folder 5

Idioma: spanish
Folder 0
Folder 1
Folder 2
Folder 3
Folder 4
Folder 5



/Users/marcospaulo/Projects/multilingual_classification/venv/lib/python3.9/site-packages/pandas/core/internals/blocks.py:940: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)
/Users/marcospaulo/Projects/multilingual_classification/venv/lib/python3.9/site-packages/pandas/core/internals/blocks.py:940: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)
/Users/marcospaulo/Projects/multilingual_classification/venv/lib/python3.9/site-packages/pandas/core/internals/blocks.py:940: VisibleDeprecationWarning:

In [7]:
ids_map

,language,ids
0,italian,"[8828_8610_000510, 659_547_000790, 643_529_000..."
1,polish,"[4228_1447_000021, 7014_6288_000494, 1329_1447..."
2,portuguese,"[10199_6390_000016, 12249_12879_002169, 3718_2..."
3,spanish,"[12332_10604_000010, 11772_11957_000011, 13690..."


Quantos áudios temos para cada idioma?

In [8]:
ids_map["count"] = ids_map["ids"].apply(len)

In [9]:
ids_map

,language,ids,count
0,italian,"[8828_8610_000510, 659_547_000790, 643_529_000...",240
1,polish,"[4228_1447_000021, 7014_6288_000494, 1329_1447...",238
2,portuguese,"[10199_6390_000016, 12249_12879_002169, 3718_2...",236
3,spanish,"[12332_10604_000010, 11772_11957_000011, 13690...",233


In [10]:
ids_map["count"].sum()

947

Vamos ler os áudios para entender as características do nosso dataset

In [11]:
ids_map = ids_map.drop(columns=["count"])

In [12]:
df_audios = ids_map.explode("ids").reset_index(drop=True)
df_audios = df_audios.rename(columns={"ids":"id"})

In [13]:
# "id", "language",  "speaker", "book", "segment"
df_audios["id_splitted"] = df_audios["id"].str.split("_")
df_audios["speaker"] = df_audios["id_splitted"].apply(lambda x : x[0])
df_audios["book"] = df_audios["id_splitted"].apply(lambda x : x[1])
df_audios["segment"] = df_audios["id_splitted"].apply(lambda x : x[2])
df_audios = df_audios.drop(columns=["id_splitted"])

In [14]:
df_audios = df_audios.astype(str)

In [15]:
df_audios

,language,id,speaker,book,segment
0,italian,8828_8610_000510,8828,8610,000510
1,italian,659_547_000790,659,547,000790
2,italian,643_529_000116,643,529,000116
3,italian,8828_8610_000321,8828,8610,000321
4,italian,659_547_000279,659,547,000279
...,...,...,...,...,...
942,spanish,6156_4006_000020,6156,4006,000020
943,spanish,101_567_000034,101,567,000034
944,spanish,8881_8550_000536,8881,8550,000536
945,spanish,6156_4006_000028,6156,4006,000028


In [16]:
# metainfo.txt (um para cada idioma): "gender", "title"
df_metainfo = pd.DataFrame()
for language in df_audios["language"].unique():
    metainfo_path = os.path.join(LANGUAGE_PATH, "metainfo.txt").format(language)
    df_aux = pd.read_csv(metainfo_path, sep="|", header=0, encoding="utf-8", dtype='str')
    df_aux.columns = df_aux.columns.str.strip().str.lower()
    df_aux["language"] = language
    df_metainfo = pd.concat([df_metainfo, df_aux])

df_metainfo = df_metainfo.apply(lambda x : x.str.strip())
df_metainfo = df_metainfo.rename(columns={"book id":"book"})

In [17]:
df_metainfo.isna().sum()

speaker      0
gender       0
partition    0
minutes      0
book         0
title        0
chapter      0
language     0
dtype: int64

In [18]:
df_metainfo.head(2)

,speaker,gender,partition,minutes,book,title,chapter,language
0,6001,F,train,31.716,10011,"Piacevoli Notti, Libro 1",Notte Prima: FAVOLA II,italian
1,6001,F,train,25.961,10011,"Piacevoli Notti, Libro 1",Notte Quinta: FAVOLA II,italian


In [19]:
df_audios = pd.merge(
    df_audios,
    df_metainfo[['speaker', 'gender', 'book', 'title', 'language']],
    on=["language", "speaker", "book"],
    how="left",
    indicator=True
)
print(df_audios["_merge"].value_counts())
df_audios = df_audios.drop_duplicates().reset_index(drop=True)
print(df_audios["_merge"].value_counts())
df_audios = df_audios.drop(columns=["_merge"])

both          8841
left_only        0
right_only       0
Name: _merge, dtype: int64
both          947
left_only       0
right_only      0
Name: _merge, dtype: int64


In [20]:
df_audios

,language,id,speaker,book,segment,gender,title
0,italian,8828_8610_000510,8828,8610,000510,F,"Novelle per un Anno, vol. 12: Il Viaggio"
1,italian,659_547_000790,659,547,000790,F,Avventure di Pinocchio
2,italian,643_529_000116,643,529,000116,F,Divina Commedia
3,italian,8828_8610_000321,8828,8610,000321,F,"Novelle per un Anno, vol. 12: Il Viaggio"
4,italian,659_547_000279,659,547,000279,F,Avventure di Pinocchio
...,...,...,...,...,...,...,...
942,spanish,6156_4006_000020,6156,4006,000020,M,Condenada y Otros Cuentos
943,spanish,101_567_000034,101,567,000034,M,Don Quijote 1
944,spanish,8881_8550_000536,8881,8550,000536,M,Aprendiz de Conspirador
945,spanish,6156_4006_000028,6156,4006,000028,M,Condenada y Otros Cuentos


Qual é a proporção de áudios masculinos e femininos por idioma?

In [21]:
men_women_proportion = df_audios.groupby("language")["gender"].value_counts().rename("counts").to_frame().reset_index()
men_women_proportion = men_women_proportion.pivot(index="language", columns="gender", values="counts")
men_women_proportion["total"] = men_women_proportion["M"] + men_women_proportion["F"]
men_women_proportion["ptc_women"] = np.round(men_women_proportion["F"] / men_women_proportion["total"] * 100, 2)
men_women_proportion["ptc_men"] = np.round(men_women_proportion["M"] / men_women_proportion["total"] * 100, 2)
men_women_proportion = men_women_proportion[['F', 'M', 'ptc_women', 'ptc_men', 'total']]
men_women_proportion

gender,F,M,ptc_women,ptc_men,total
language,,,,,
italian,116,124,48.33,51.67,240
polish,118,120,49.58,50.42,238
portuguese,119,117,50.42,49.58,236
spanish,115,118,49.36,50.64,233


In [22]:
# audio
audios = []
lengths = []
srs = []
for i, row in df_audios.iterrows():
    print(i)
    print("Idioma: {}".format(row["language"]))
    print("ID: {}".format(row["id"]))
    
    audio_path = AUDIO_PATH.format(row["language"], row["speaker"], row["book"], row["id"])
    audio, sr = librosa.load(audio_path, sr=16000, mono=True) # sr é fornecido no README.md
    length = len(audio)
    print("Leu {} com sucesso!".format(audio_path))
    
    audios.append(audio)
    srs.append(sr)
    lengths.append(length)
    
    print()

0
Idioma: italian
ID: 8828_8610_000510
Leu ../data/mls/mls_italian/train/audio/8828/8610/8828_8610_000510.flac com sucesso!

1
Idioma: italian
ID: 659_547_000790
Leu ../data/mls/mls_italian/train/audio/659/547/659_547_000790.flac com sucesso!

2
Idioma: italian
ID: 643_529_000116
Leu ../data/mls/mls_italian/train/audio/643/529/643_529_000116.flac com sucesso!

3
Idioma: italian
ID: 8828_8610_000321
Leu ../data/mls/mls_italian/train/audio/8828/8610/8828_8610_000321.flac com sucesso!

4
Idioma: italian
ID: 659_547_000279
Leu ../data/mls/mls_italian/train/audio/659/547/659_547_000279.flac com sucesso!

5
Idioma: italian
ID: 643_529_000078
Leu ../data/mls/mls_italian/train/audio/643/529/643_529_000078.flac com sucesso!

6
Idioma: italian
ID: 8828_8610_000293
Leu ../data/mls/mls_italian/train/audio/8828/8610/8828_8610_000293.flac com sucesso!

7
Idioma: italian
ID: 659_547_000214
Leu ../data/mls/mls_italian/train/audio/659/547/659_547_000214.flac com sucesso!

8
Idioma: italian
ID: 643_529_

Leu ../data/mls/mls_italian/train/audio/1595/4194/1595_4194_000519.flac com sucesso!

72
Idioma: italian
ID: 6299_5281_000283
Leu ../data/mls/mls_italian/train/audio/6299/5281/6299_5281_000283.flac com sucesso!

73
Idioma: italian
ID: 4705_9630_000143
Leu ../data/mls/mls_italian/train/audio/4705/9630/4705_9630_000143.flac com sucesso!

74
Idioma: italian
ID: 1595_3311_000265
Leu ../data/mls/mls_italian/train/audio/1595/3311/1595_3311_000265.flac com sucesso!

75
Idioma: italian
ID: 6299_5281_000306
Leu ../data/mls/mls_italian/train/audio/6299/5281/6299_5281_000306.flac com sucesso!

76
Idioma: italian
ID: 4705_9630_000032
Leu ../data/mls/mls_italian/train/audio/4705/9630/4705_9630_000032.flac com sucesso!

77
Idioma: italian
ID: 1595_4194_000588
Leu ../data/mls/mls_italian/train/audio/1595/4194/1595_4194_000588.flac com sucesso!

78
Idioma: italian
ID: 6299_5281_000026
Leu ../data/mls/mls_italian/train/audio/6299/5281/6299_5281_000026.flac com sucesso!

79
Idioma: italian
ID: 4705_9630

Leu ../data/mls/mls_italian/train/audio/8461/7877/8461_7877_000220.flac com sucesso!

138
Idioma: italian
ID: 6807_7720_000251
Leu ../data/mls/mls_italian/train/audio/6807/7720/6807_7720_000251.flac com sucesso!

139
Idioma: italian
ID: 8842_8610_000096
Leu ../data/mls/mls_italian/train/audio/8842/8610/8842_8610_000096.flac com sucesso!

140
Idioma: italian
ID: 8461_7877_000045
Leu ../data/mls/mls_italian/train/audio/8461/7877/8461_7877_000045.flac com sucesso!

141
Idioma: italian
ID: 12598_9630_000036
Leu ../data/mls/mls_italian/train/audio/12598/9630/12598_9630_000036.flac com sucesso!

142
Idioma: italian
ID: 1595_5616_001101
Leu ../data/mls/mls_italian/train/audio/1595/5616/1595_5616_001101.flac com sucesso!

143
Idioma: italian
ID: 644_2532_000345
Leu ../data/mls/mls_italian/train/audio/644/2532/644_2532_000345.flac com sucesso!

144
Idioma: italian
ID: 12598_9630_000011
Leu ../data/mls/mls_italian/train/audio/12598/9630/12598_9630_000011.flac com sucesso!

145
Idioma: italian
ID

Leu ../data/mls/mls_italian/train/audio/8842/8610/8842_8610_000165.flac com sucesso!

203
Idioma: italian
ID: 8828_8610_000603
Leu ../data/mls/mls_italian/train/audio/8828/8610/8828_8610_000603.flac com sucesso!

204
Idioma: italian
ID: 7230_6241_000043
Leu ../data/mls/mls_italian/train/audio/7230/6241/7230_6241_000043.flac com sucesso!

205
Idioma: italian
ID: 8842_8795_000356
Leu ../data/mls/mls_italian/train/audio/8842/8795/8842_8795_000356.flac com sucesso!

206
Idioma: italian
ID: 8828_7877_000015
Leu ../data/mls/mls_italian/train/audio/8828/7877/8828_7877_000015.flac com sucesso!

207
Idioma: italian
ID: 7230_6241_000033
Leu ../data/mls/mls_italian/train/audio/7230/6241/7230_6241_000033.flac com sucesso!

208
Idioma: italian
ID: 8842_8610_000167
Leu ../data/mls/mls_italian/train/audio/8842/8610/8842_8610_000167.flac com sucesso!

209
Idioma: italian
ID: 8828_7877_000113
Leu ../data/mls/mls_italian/train/audio/8828/7877/8828_7877_000113.flac com sucesso!

210
Idioma: italian
ID: 7

Leu ../data/mls/mls_polish/train/audio/3553/1447/3553_1447_000013.flac com sucesso!

278
Idioma: polish
ID: 4228_1447_000022
Leu ../data/mls/mls_polish/train/audio/4228/1447/4228_1447_000022.flac com sucesso!

279
Idioma: polish
ID: 1329_1447_000028
Leu ../data/mls/mls_polish/train/audio/1329/1447/1329_1447_000028.flac com sucesso!

280
Idioma: polish
ID: 7014_6834_000084
Leu ../data/mls/mls_polish/train/audio/7014/6834/7014_6834_000084.flac com sucesso!

281
Idioma: polish
ID: 4228_1447_000041
Leu ../data/mls/mls_polish/train/audio/4228/1447/4228_1447_000041.flac com sucesso!

282
Idioma: polish
ID: 1329_1447_000003
Leu ../data/mls/mls_polish/train/audio/1329/1447/1329_1447_000003.flac com sucesso!

283
Idioma: polish
ID: 7014_6254_000032
Leu ../data/mls/mls_polish/train/audio/7014/6254/7014_6254_000032.flac com sucesso!

284
Idioma: polish
ID: 4228_1447_000048
Leu ../data/mls/mls_polish/train/audio/4228/1447/4228_1447_000048.flac com sucesso!

285
Idioma: polish
ID: 1329_1447_000015


...

Leu ../data/mls/mls_portuguese/train/audio/9351/9018/9351_9018_001311.flac com sucesso!

624
Idioma: portuguese
ID: 4341_3604_000000
Leu ../data/mls/mls_portuguese/train/audio/4341/3604/4341_3604_000000.flac com sucesso!

625
Idioma: portuguese
ID: 12428_13396_000008
Leu ../data/mls/mls_portuguese/train/audio/12428/13396/12428_13396_000008.flac com sucesso!

626
Idioma: portuguese
ID: 9351_9280_001125
Leu ../data/mls/mls_portuguese/train/audio/9351/9280/9351_9280_001125.flac com sucesso!

627
Idioma: portuguese
ID: 4341_3604_000001
Leu ../data/mls/mls_portuguese/train/audio/4341/3604/4341_3604_000001.flac com sucesso!

628
Idioma: portuguese
ID: 12428_13396_000017
Leu ../data/mls/mls_portuguese/train/audio/12428/13396/12428_13396_000017.flac com sucesso!

629
Idioma: portuguese
ID: 9351_9280_001742
Leu ../data/mls/mls_portuguese/train/audio/9351/9280/9351_9280_001742.flac com sucesso!

630
Idioma: portuguese
ID: 4341_2564_000002
Leu ../data/mls/mls_portuguese/train/audio/4341/2564/4341

Leu ../data/mls/mls_portuguese/train/audio/9958/6390/9958_6390_000002.flac com sucesso!

690
Idioma: portuguese
ID: 12249_12879_000927
Leu ../data/mls/mls_portuguese/train/audio/12249/12879/12249_12879_000927.flac com sucesso!

691
Idioma: portuguese
ID: 9958_6390_000000
Leu ../data/mls/mls_portuguese/train/audio/9958/6390/9958_6390_000000.flac com sucesso!

692
Idioma: portuguese
ID: 12249_12879_001771
Leu ../data/mls/mls_portuguese/train/audio/12249/12879/12249_12879_001771.flac com sucesso!

693
Idioma: portuguese
ID: 9958_6390_000001
Leu ../data/mls/mls_portuguese/train/audio/9958/6390/9958_6390_000001.flac com sucesso!

694
Idioma: portuguese
ID: 12249_10229_000456
Leu ../data/mls/mls_portuguese/train/audio/12249/10229/12249_10229_000456.flac com sucesso!

695
Idioma: portuguese
ID: 5677_4807_000490
Leu ../data/mls/mls_portuguese/train/audio/5677/4807/5677_4807_000490.flac com sucesso!

696
Idioma: portuguese
ID: 2959_2564_000018
Leu ../data/mls/mls_portuguese/train/audio/2959/256

Leu ../data/mls/mls_spanish/train/audio/12921/12700/12921_12700_000016.flac com sucesso!

755
Idioma: spanish
ID: 13690_14558_000067
Leu ../data/mls/mls_spanish/train/audio/13690/14558/13690_14558_000067.flac com sucesso!

756
Idioma: spanish
ID: 8304_7613_000002
Leu ../data/mls/mls_spanish/train/audio/8304/7613/8304_7613_000002.flac com sucesso!

757
Idioma: spanish
ID: 12921_12700_000021
Leu ../data/mls/mls_spanish/train/audio/12921/12700/12921_12700_000021.flac com sucesso!

758
Idioma: spanish
ID: 13690_14644_000106
Leu ../data/mls/mls_spanish/train/audio/13690/14644/13690_14644_000106.flac com sucesso!

759
Idioma: spanish
ID: 8304_7613_000052
Leu ../data/mls/mls_spanish/train/audio/8304/7613/8304_7613_000052.flac com sucesso!

760
Idioma: spanish
ID: 12921_12700_000011
Leu ../data/mls/mls_spanish/train/audio/12921/12700/12921_12700_000011.flac com sucesso!

761
Idioma: spanish
ID: 13690_11991_000027
Leu ../data/mls/mls_spanish/train/audio/13690/11991/13690_11991_000027.flac com s

Leu ../data/mls/mls_spanish/train/audio/11228/10604/11228_10604_000014.flac com sucesso!

822
Idioma: spanish
ID: 2262_2162_000061
Leu ../data/mls/mls_spanish/train/audio/2262/2162/2262_2162_000061.flac com sucesso!

823
Idioma: spanish
ID: 10982_9310_000022
Leu ../data/mls/mls_spanish/train/audio/10982/9310/10982_9310_000022.flac com sucesso!

824
Idioma: spanish
ID: 11228_10604_000008
Leu ../data/mls/mls_spanish/train/audio/11228/10604/11228_10604_000008.flac com sucesso!

825
Idioma: spanish
ID: 2262_2162_000000
Leu ../data/mls/mls_spanish/train/audio/2262/2162/2262_2162_000000.flac com sucesso!

826
Idioma: spanish
ID: 12921_12700_000007
Leu ../data/mls/mls_spanish/train/audio/12921/12700/12921_12700_000007.flac com sucesso!

827
Idioma: spanish
ID: 11772_11957_000003
Leu ../data/mls/mls_spanish/train/audio/11772/11957/11772_11957_000003.flac com sucesso!

828
Idioma: spanish
ID: 8304_7613_000179
Leu ../data/mls/mls_spanish/train/audio/8304/7613/8304_7613_000179.flac com sucesso!



Leu ../data/mls/mls_spanish/train/audio/11228/10604/11228_10604_000004.flac com sucesso!

891
Idioma: spanish
ID: 6615_11957_000058
Leu ../data/mls/mls_spanish/train/audio/6615/11957/6615_11957_000058.flac com sucesso!

892
Idioma: spanish
ID: 8881_8550_000594
Leu ../data/mls/mls_spanish/train/audio/8881/8550/8881_8550_000594.flac com sucesso!

893
Idioma: spanish
ID: 11228_10604_000007
Leu ../data/mls/mls_spanish/train/audio/11228/10604/11228_10604_000007.flac com sucesso!

894
Idioma: spanish
ID: 6615_11957_000078
Leu ../data/mls/mls_spanish/train/audio/6615/11957/6615_11957_000078.flac com sucesso!

895
Idioma: spanish
ID: 8881_8550_000167
Leu ../data/mls/mls_spanish/train/audio/8881/8550/8881_8550_000167.flac com sucesso!

896
Idioma: spanish
ID: 11228_10604_000013
Leu ../data/mls/mls_spanish/train/audio/11228/10604/11228_10604_000013.flac com sucesso!

897
Idioma: spanish
ID: 6615_11957_000068
Leu ../data/mls/mls_spanish/train/audio/6615/11957/6615_11957_000068.flac com sucesso!



In [23]:
df_audios["audio"] = audios
df_audios["length"] = lengths
df_audios["sr"] = srs

In [24]:
df_audios

,language,id,speaker,book,segment,gender,title,audio,length,sr
0,italian,8828_8610_000510,8828,8610,000510,F,"Novelle per un Anno, vol. 12: Il Viaggio","[0.0, 3.0517578e-05, 3.0517578e-05, 6.1035156e...",318560,16000
1,italian,659_547_000790,659,547,000790,F,Avventure di Pinocchio,"[0.0, 3.0517578e-05, 0.0, 3.0517578e-05, 3.051...",186880,16000
2,italian,643_529_000116,643,529,000116,F,Divina Commedia,"[-0.007232666, -0.00491333, -0.0054016113, -0....",314560,16000
3,italian,8828_8610_000321,8828,8610,000321,F,"Novelle per un Anno, vol. 12: Il Viaggio","[0.0, 3.0517578e-05, 0.0, 0.0, 3.0517578e-05, ...",229920,16000
4,italian,659_547_000279,659,547,000279,F,Avventure di Pinocchio,"[0.0, 0.0, 0.0, -3.0517578e-05, -9.1552734e-05...",197440,16000
...,...,...,...,...,...,...,...,...,...,...
942,spanish,6156_4006_000020,6156,4006,000020,M,Condenada y Otros Cuentos,"[0.0, 6.1035156e-05, 9.1552734e-05, 6.1035156e...",250880,16000
943,spanish,101_567_000034,101,567,000034,M,Don Quijote 1,"[0.0020751953, 0.0024108887, 0.002166748, 0.00...",297600,16000
944,spanish,8881_8550_000536,8881,8550,000536,M,Aprendiz de Conspirador,"[0.00076293945, 0.00064086914, 0.0002746582, 0...",162080,16000
945,spanish,6156_4006_000028,6156,4006,000028,M,Condenada y Otros Cuentos,"[-0.00036621094, -0.0004272461, -0.00045776367...",238240,16000


In [25]:
gc.collect()

4

Quanto tempo de áudio temos para cada idioma?

In [26]:
df_audios["duration_s"] = df_audios.apply(lambda x : librosa.get_duration(y=x["audio"], sr=x["sr"]), axis=1)

In [27]:
df_audios.head()

,language,id,speaker,book,segment,gender,title,audio,length,sr,duration_s
0,italian,8828_8610_000510,8828,8610,000510,F,"Novelle per un Anno, vol. 12: Il Viaggio","[0.0, 3.0517578e-05, 3.0517578e-05, 6.1035156e...",318560,16000,19.91
1,italian,659_547_000790,659,547,000790,F,Avventure di Pinocchio,"[0.0, 3.0517578e-05, 0.0, 3.0517578e-05, 3.051...",186880,16000,11.68
2,italian,643_529_000116,643,529,000116,F,Divina Commedia,"[-0.007232666, -0.00491333, -0.0054016113, -0....",314560,16000,19.66
3,italian,8828_8610_000321,8828,8610,000321,F,"Novelle per un Anno, vol. 12: Il Viaggio","[0.0, 3.0517578e-05, 0.0, 0.0, 3.0517578e-05, ...",229920,16000,14.37
4,italian,659_547_000279,659,547,000279,F,Avventure di Pinocchio,"[0.0, 0.0, 0.0, -3.0517578e-05, -9.1552734e-05...",197440,16000,12.34


In [28]:
seconds_total = df_audios.groupby("language")["duration_s"].sum()
seconds_total

language
italian       3559.770125
polish        3575.904312
portuguese    3581.950375
spanish       3580.304125
Name: duration_s, dtype: float64

In [29]:
minutes = seconds_total / 60
seconds = seconds_total - (minutes).astype(int) * 60

In [30]:
pd.concat([minutes.astype(int).rename("minutes").to_frame(), seconds.astype(int).rename("seconds").to_frame()], axis=1)

,minutes,seconds
language,,
italian,59,19
polish,59,35
portuguese,59,41
spanish,59,40


Salvo o dataframe final

In [31]:
df_audios.to_csv("../data/audios.csv")